# Excel 파일 처리 테스트

이 노트북은 Excel 파일을 읽어 CSV로 변환하는 기능을 테스트합니다.

## 설정 사항
- 프로젝트 루트: `/home/cjkim/illumina/aem_qa_system`
- 대상 파일 경로: `data/1_input/source_excels/glossary`

In [ ]:
# src/utils/excel_processor.py

import pandas as pd
import os
import glob
import numpy as np
from src.config import PROJECT_ROOT


In [ ]:
from multiprocessing.util import is_exiting


for root, dirs, files in os.walk(PROJECT_ROOT):
    for file in files:
        if file.endswith("xlsx"):
            print(os.path.join(root,file))
        elif dirs:
            for _ in dirs:
                print(f"{root}/{_}/")
        else:
            print("\n")

In [104]:
def _load_and_tag_excels_from_folder(folder_path: str) -> pd.DataFrame | None:
    """
    지정된 폴더의 모든 Excel을 읽고, 'Source_File' 컬럼을 추가한 후,
    하나의 데이터프레임으로 합쳐서 반환합니다.
    """
    excel_files = glob.glob(os.path.join(folder_path, "*.xlsx"))
    if not excel_files:
        print(f"⚠️ 경고: '{folder_path}' 폴더에 Excel 파일이 없습니다.")
        return None

    all_dfs = []
    for file_path in excel_files:
        df = pd.read_excel(file_path)
        df['Source_File'] = os.path.basename(file_path)
        all_dfs.append(df)
    
    return pd.concat(all_dfs, ignore_index=True)
# ▲▲▲▲▲ [신규] 중복 로직을 처리하는 내부 헬퍼 함수 ▲▲▲▲▲

In [ ]:
glossary_path = os.path.join(PROJECT_ROOT, "data/1_input/source_excels/glossary")
df = _load_and_tag_excels_from_folder(glossary_path)
df[['en-US', 'ko-KR', 'Source_File']]

In [ ]:
# 디렉토리가 존재하면 내용 확인
def check_dir_content(dir_path, name):
    if os.path.exists(dir_path):
        print(f"\n{name} 디렉토리 내용:")
        entries = os.listdir(dir_path)
        
        print(f"총 {len(entries)}개 항목 발견")
        
        # 파일과 디렉토리 분류
        files = []
        dirs = []
        
        for entry in entries:
            full_path = os.path.join(dir_path, entry)
            if os.path.isdir(full_path):
                dirs.append(entry)
            else:
                files.append(entry)
        
        # 디렉토리 목록 출력
        if dirs:
            print(f"\n디렉토리 ({len(dirs)}개):")
            for directory in dirs:
                dir_path_full = os.path.join(dir_path, directory)
                dir_contents = os.listdir(dir_path_full)
                print(f" - 📁 {directory} ({len(dir_contents)}개 항목)")
        
        # 파일 목록 출력
        if files:
            print(f"\n파일 ({len(files)}개):")
            for file in files:
                ext = os.path.splitext(file)[1]
                if ext in ['.xlsx', '.xls']:
                    print(f" - 📊 {file}")
                else:
                    print(f" - 📄 {file}")
        
        # Excel 파일만 필터링
        excel_files = [f for f in files if f.endswith(('.xlsx', '.xls'))]
        if excel_files:
            print(f"\n{name}에서 발견된 Excel 파일: {excel_files}")
    else:
        print(f"\n{name} 디렉토리가 존재하지 않습니다.")

# 양쪽 경로 모두 확인
check_dir_content(PROJECT_ROOT, "프로젝트 내 경로")
# check_dir_content(home_glossary_path, "홈 기준 경로")

In [ ]:
def process_glossary_excel_to_csv(input_folder_path: str,
                                  output_csv_path: str,
                                  source_col: str,
                                  target_col: str):
    """
    [단순 변환용] 용어집 폴더의 Excel 파일들을 읽어 하나의 CSV로 통합합니다.
    """
    # ... (이전 제안과 동일한 간단한 처리 로직)
    print(f"🔄 용어집 폴더 처리 중: '{os.path.basename(input_folder_path)}'...")
    try:
        excel_files = glob.glob(os.path.join(input_folder_path, "*.xlsx"))
        if not excel_files:
            print(f"⚠️ 경고: '{input_folder_path}' 폴더에 Excel 파일이 없습니다.")
            return False
    
        
        df = pd.concat([pd.read_excel(f) for f in excel_files], ignore_index=True) # df에 맞게 새로이 index생성
        df_processed = df[[source_col, target_col]].copy()
        df_processed.rename(columns={source_col: 'Source', target_col: 'Target'}, inplace=True) # inplace를 사용해서 원본df를 직접 수정
        df_processed.dropna(subset=['Source', 'Target'], inplace=True)
        df_processed.drop_duplicates(subset=['Source', 'Target'], inplace=True)
        
        os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
        df_processed.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
        
        print(f"✅ 완료: '{os.path.basename(output_csv_path)}' 생성 ({len(df_processed)}개 행)")
        return True
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return False

In [ ]:
# 프로젝트 루트를 기준으로 경로 설정하기
import os
import glob

# 1. 프로젝트 루트 경로 설정
project_root = "/home/cjkim/illumina/aem_qa_system"
print(f"프로젝트 루트: {project_root}")

# 2. 상대 경로 설정
rel_path = "data/1_input/source_excels/glossary"
full_path = os.path.join(project_root, rel_path)
print(f"상대 경로: {rel_path}")
print(f"전체 경로: {full_path}")

# 3. 파일 찾기
excel_files = glob.glob(os.path.join(full_path, "*.xlsx"))
print(f"찾은 Excel 파일 수: {len(excel_files)}")

if excel_files:
    print("\n파일 목록:")
    for file in excel_files:
        # 프로젝트 루트 기준 상대 경로로 표시
        rel_file_path = os.path.relpath(file, project_root)
        print(f" - {rel_file_path}")
        
    # 첫 번째 파일을 사용하여 process_excel_folder_to_csv 함수 실행 예시
    print("\n프로젝트 루트 기준 상대 경로로 함수를 호출할 경우:")
    print(f"input_folder_path='{rel_path}'")
    print(f"output_csv_path='{os.path.join(rel_path, 'processed_output.csv')}'")
else:
    print("Excel 파일을 찾을 수 없습니다.")

In [ ]:
def process_excel_folder_to_csv(input_folder_path: str,
                                output_csv_path: str,
                                source_col: str,
                                target_col: str):
    """
    지정된 폴더 안의 모든 Excel 파일을 읽어 'Source_File' 출처를 포함한 CSV로 통합합니다.
    
    Args:
        input_folder_path: Excel 파일이 있는 폴더 경로
        output_csv_path: 저장할 CSV 파일 경로
        source_col: 원본 언어 컬럼명
        target_col: 대상 언어 컬럼명
        
    Returns:
        bool: 성공 시 True, 실패 시 False
    """
    print(f"🔄 폴더 처리 중: '{os.path.basename(input_folder_path)}'...")
    try:
        # 1. Excel 파일 찾기
        excel_files = glob.glob(os.path.join(input_folder_path, "*.xlsx"))
        if not excel_files:
            print(f"⚠️ 경고: '{input_folder_path}' 폴더에 Excel 파일이 없습니다.")
            return False
        
        print(f"📊 처리할 Excel 파일 수: {len(excel_files)}")
        
        # 2. 각 Excel 파일 처리
        all_dfs = []
        for file_path in excel_files:
            print(f"📄 파일 처리 중: {os.path.basename(file_path)}")
            df = pd.read_excel(file_path)
            
            # 원본 파일명 추가
            df['Source_File'] = os.path.basename(file_path)
            all_dfs.append(df)
        
        # 3. 모든 데이터프레임 합치기
        combined_df = pd.concat(all_dfs, ignore_index=True)
        print(f"👉 모든 파일 병합 완료: 총 {len(combined_df)}개 행")
        
        # 4. 컬럼 선택 및 이름 변경
        df_processed = combined_df[[source_col, target_col, 'Source_File']].copy()
        df_processed.rename(columns={source_col: 'Source', target_col: 'Target'}, inplace=True)
        
        # 5. 데이터 정제
        # - NA 값 제거
        before_na_drop = len(df_processed)
        df_processed.dropna(subset=['Source', 'Target'], inplace=True)
        after_na_drop = len(df_processed)
        print(f"🧹 NA 값 제거: {before_na_drop - after_na_drop}개 행 제거됨")
        
        # - 중복 제거
        before_dedup = len(df_processed)
        df_processed.drop_duplicates(subset=['Source', 'Target'], inplace=True)
        after_dedup = len(df_processed)
        print(f"🧹 중복 제거: {before_dedup - after_dedup}개 행 제거됨")
        
        # 6. CSV로 저장
        # - 출력 디렉토리 생성
        os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
        
        # - CSV 저장
        df_processed.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
        
        print(f"✅ 완료: '{os.path.basename(output_csv_path)}' 생성 ({len(df_processed)}개 행)")
        return True
        
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
        return False

In [ ]:
# 프로젝트 루트 기준으로 process_excel_folder_to_csv 함수 실행
import os
import sys

# 1. 프로젝트 루트 설정 및 이를 sys.path에 추가
project_root = "/home/cjkim/illumina/aem_qa_system"
if project_root not in sys.path:
    sys.path.append(project_root)  # 프로젝트 모듈 import를 위해 필요할 경우

# 2. 상대 경로 설정
input_rel_path = "data/1_input/source_excels/glossary"
output_rel_path = "data/1_input/source_excels/glossary/test.csv"

# 3. 전체 경로로 변환 (함수 내부에서 필요한 경우)
input_full_path = os.path.join(project_root, input_rel_path)
output_full_path = os.path.join(project_root, output_rel_path)

print(f"프로젝트 루트: {project_root}")
print(f"입력 상대 경로: {input_rel_path}")
print(f"출력 상대 경로: {output_rel_path}")

# 4. 디렉토리 존재 확인
if os.path.exists(input_full_path):
    print(f"\n✅ 입력 경로가 존재합니다: {input_rel_path}")
    
    # 5. 디렉토리 내용 출력 (파일 이름만)
    files = os.listdir(input_full_path)
    excel_files = [f for f in files if f.endswith('.xlsx')]
    
    print(f"Excel 파일 수: {len(excel_files)}")
    if excel_files:
        print(f"Excel 파일 목록:")
        for excel_file in excel_files:
            print(f" - {excel_file}")
    
    # 6. 함수 실행
    result = process_excel_folder_to_csv(
        input_folder_path=input_full_path,  # 전체 경로 사용
        output_csv_path=output_full_path,   # 전체 경로 사용
        source_col="en-US",
        target_col="ko-KR"
    )
    
    print(f"\n실행 결과: {'성공 ✅' if result else '실패 ❌'}")
else:
    print(f"❌ 입력 경로가 존재하지 않습니다: {input_rel_path}")

In [ ]:


def process_tm_excel_to_csv(input_folder_path: str,
                            output_csv_path: str,
                            source_col: str,
                            # ... (이하 파라미터 동일)
                            ):
    """
    [조건부 업데이트용] TM 폴더의 Excel 파일들을 읽어,
    'Source_File' 출처를 포함하여 하나의 CSV로 통합합니다.
    """
    print(f"🔄 TM 폴더 처리 중 (업데이트 로직 적용): '{os.path.basename(input_folder_path)}'...")
    try:
        # ... (Excel 파일 읽기 및 'Source_File' 컬럼 추가 로직은 위와 동일)
        
        all_dfs = []
        for file_path in glob.glob(os.path.join(input_folder_path, "*.xlsx")):
            df = pd.read_excel(file_path)
            df['Source_File'] = os.path.basename(file_path)
            all_dfs.append(df)
        
        df = pd.concat(all_dfs, ignore_index=True)

        # 1. 최종 Target 결정 로직
        df['Final_Target'] = np.where(...)

        # 2. 원본(en-US)과 최종 Target 쌍 생성
        df1 = df[[source_col, 'Final_Target', 'Source_File']].copy()
        df1.rename(columns={source_col: 'Source', 'Final_Target': 'Target'}, inplace=True)
        
        # 3. 업데이트(en-US1)와 최종 Target 쌍 생성
        df2 = df[df[source_col_updated].notna()][[source_col_updated, 'Final_Target', 'Source_File']].copy()
        df2.rename(columns={source_col_updated: 'Source', 'Final_Target': 'Target'}, inplace=True)

        # 4. 두 쌍의 목록을 하나로 합치고 정제
        final_df = pd.concat([df1, df2], ignore_index=True)
        final_df.dropna(subset=['Source', 'Target'], inplace=True)
        # 이제 Source, Target, Source_File 세 가지가 모두 같아야 중복으로 간주
        final_df.drop_duplicates(subset=['Source', 'Target', 'Source_File'], inplace=True)
        
        # 5. CSV로 저장
        final_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

        print(f"✅ 완료: '{os.path.basename(output_csv_path)}' 생성 ({len(final_df)}개 행)")
        return True
    except Exception as e:
        # ...
        return False